In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
## GLADE+ galaxy catalogue:
## Column 1 : GLADEno;          GLADE+ catalog number 
## Column 3 : GWGCname;         Name in the GWGC catalog
## Column 9 : RA                Right ascension in degrees
## Column 10: Dec               Declination in degrees
## Column 14: B_Abs             Absolute B magnitude
## Column 33: d_L               Luminosity distance in Mpc units
## Column 34: d_Lerr           Error of luminosity distance in Mpc units
## Column 36: Mst               Stellar mass in 10^10 M_Sun units
## Column 37: Mst_err           Absolute error of stellar mass in 10^10 M_Sun units
## Column 39: Merger_rate       Base-10 logarithm of estimated BNS merger rate in the galaxy in Gyr^-1 units
## Column 40: Merger_rate_error Absolute error of estimated BNS merger rate in the galaxy
## For GLADE+ 2.4:
## Column 1 : GLADEno;          GLADE+ catalog number 
## Column 2 : GWGCname;         Name in the GWGC catalog
## Column 7 : ra                Right ascension in degrees
## Column 8: dec               Declination in degrees
## Column 14: M_b             Absolute B magnitude
## Column 9: dis               Luminosity distance in Mpc units

In [5]:
# injections
RA_inj = 146.182916666667 * np.pi/180 # rads, 9.745527777778 hours
dec_inj = 9.610000000000 * np.pi/180 # rads
dis_inj = 9400.000000000000 / 1000 # MPc

# read samples
test_bin_size = 0
plot_key = 1

data_name = ['../../../GW170817/high_spin_PhenomPNRT_posterior_samples.dat']



In [ ]:
for idata in range(len(data_name)):
    # read the GW posterior data
    sample_data = data_name[idata]
    pos = np.loadtxt(sample_data, dtype = 'str')
    pos = np.delete(pos, 0, axis=0)
    n_post_sample = len(pos[:, 2])
    
    
    
    ## load the galaxy catalogue GC
    GC = np.loadtxt('../../../GC/GLADE_2.4.txt', dtype = 'str');
    ra = GC[:,6]; 
    dec = GC[:,7];
    M_b = GC[:,13];
    dis = GC[:,8];
    raa = ra * 15 * np.pi / 180  # rads
    decc = dec * np.pi / 180  # rads
    likelihood_B = np.abs(M_b)  # f function
    del GC # to save the memory
    # 3d bins by galaxy locations
    if test_bin_size == 1:
        dis_size = np.arange(0.01, 2.01, 0.01)  # real size is dis_size*dis
        decc_err = np.arange(0.01, 0.11, 0.01)
        raa_err = decc_err
    else:
        dis_size = np.array([0.1])  # real size is dis_size*dis
        decc_err = np.array([0.01])
        raa_err = decc_err

    for ds in range(len(dis_size)):
        for dra in range(len(raa_err)):
            n_pos_in_bin = np.zeros(length_d)
            ga_count_g_b = np.zeros(length_d)

            for gg in range(length_d):
                dis_err = dis_size[ds] * dis[gg]
                dis_low = dis[gg] - dis_err
                dis_high = dis[gg] + dis_err
                raa_low = raa[gg] - raa_err[dra]
                raa_high = raa[gg] + raa_err[dra]
                decc_low = decc[gg] - decc_err[dra]
                decc_high = decc[gg] + decc_err[dra]

                ga_pos = np.where(
                    (dis_low <= pos[:, 12]) &
                    (pos[:, 12] <= dis_high) &
                    (raa_low <= pos[:, 1]) &
                    (pos[:, 1] <= raa_high) &
                    (decc_low <= pos[:, 17]) &
                    (pos[:, 17] < decc_high)
                )[0]  # which galaxy is in each cube
                #n_pos_in_bin[gg] = imcomplete * len(ga_pos) / n_post_sample  # normalized
                n_pos_in_bin[gg] = len(ga_pos) / n_post_sample
                ga_count_g_b[gg] = n_pos_in_bin[gg] * likelihood_B[gg] / dis_err

            rank_count_b = np.sort(ga_count_g_b)[::-1]
            id_cont_b = np.argsort(ga_count_g_b)[::-1]
            rank_b = np.sort(likelihood_B)[::-1]
            id_b = np.argsort(likelihood_B)[::-1]

            n_rank_count_b = len(rank_count_b)
            post_total = rank_count_b[:n_rank_count_b] / np.sum(rank_count_b)

            post_total_no_zero = post_total[np.nonzero(post_total)]
            length_post_total = len(post_total_no_zero)

            post_no_zero_filename = 'post_no_zero_galaxy_bin_.txt'

            ga_count_b_inj_id = np.where((np.abs(dis - dis_inj) < 0.00001) & (np.abs(raa - RA_inj) < 0.00001) & (np.abs(decc - dec_inj) < 0.0001))[0]
            ##
            if test_bin_size == 0 and plot_key == 1:
                plt.figure(idata)
                ax = plt.axes(projection='3d')
                ax.scatter(dis_inj, RA_inj, dec_inj, c='r', marker='*', s=6)
                # n_post_use = 256  # the min number of count
                ax.scatter(pos[:, 12], pos[:, 1], pos[:, 17], c='b', marker='.', s=4)
                for ff in range(3):
                    ax.scatter(dis[id_cont_b[ff]], ra[id_cont_b[ff]]*15*np.pi/180, dec[id_cont_b[ff]]*np.pi/180, c='r', marker='x', s=6)
                    ax.scatter(dis[id_cont_b[ff]], ra[id_cont_b[ff]]*15*np.pi/180, dec[id_cont_b[ff]]*np.pi/180, c='k', marker='o', s=(10-ff)**2)
                ax.scatter(dis_inj, RA_inj, dec_inj, c='r', marker='*', s=6)
                ax.set_xlabel('dis')
                ax.set_ylabel('R.A.')
                ax.set_zlabel('Dec')
                ax.legend(['injection', 'post samples', 'galaxy in bins'])
                post_name = f'galaxy_count_b_in_g_bins_event_{data_name[idata]}.eps'
                plt.savefig(post_name, format='eps')

            post_total = rank_count_b[0:n_rank_count_b] / np.sum(rank_count_b)
            length_post_total = np.sum(post_total != 0)
            post_total_no_zero = post_total[0:length_post_total]
            post_no_zero_filename = 'post_no_zero_galaxy_bin_.txt'

            ga_count_b_inj_id = np.where((np.abs(dis-dis_inj) < 0.00001) & (np.abs(raa-RA_inj) < 0.00001) & (np.abs(decc-dec_inj) < 0.0001))[0]

            post_inj[ds, dra] = ga_count_g_b[ga_count_b_inj_id] / np.sum(rank_count_b)
            if np.sum(post_total == post_inj[ds, dra]) == 1:
                ga_rank[ds, dra] = np.where(post_total == post_inj[ds, dra])[0][0]
            else:
                ga_rank[ds, dra] = 0



In [ ]:
if test_bin_size:
    plt.figure(22)
    plt.hold(True)
    plt.plot(dis_size, post_inj[:,0], 'r', markersize=6)
    plt.plot(dis_size, post_inj[:,1], 'k', markersize=6)
    plt.plot(dis_size, post_inj[:,2], 'g', markersize=6)
    plt.plot(dis_size, post_inj[:,4], 'b', markersize=6)
    plt.plot(dis_size, post_inj[:,9], 'r:', markersize=6)
    plt.xlabel('distant size fraction')
    plt.ylabel('posterior probability ')
    plt.legend(('ra err 0.01', 'ra err 0.02', 'ra err 0.03', 'ra err 0.05', 'ra err 1'))
    bins_name = 'ra_dis_size_pro' + str(data_name[idata]) + '.eps'
    plt.savefig(bins_name, format='eps')
